# Baseline

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.model_selection import KFold
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from utils import *

## Helper Function

In [2]:
def custom_collate(batch):
    images, labels = zip(*batch)
    return torch.stack(images), torch.tensor(labels)[:, None]  # Reshape labels to (batch_size, 1)

### Set your dataset directory

In [3]:
data_dir = '../../data/BiteCount/salient_poses/' 

# Define transformations for training set and validation set
train_transforms = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset with training transformations
dataset = datasets.ImageFolder(data_dir, transform=train_transforms)
dataset_size = len(dataset)
class_names = dataset.classes

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create data loaders for train and validation datasets
batch_size = 192
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize model, criterion, optimizer
num_epochs = 25
patience = 5

## Train the model

In [4]:
# Perform 5-Fold Cross Validation
fold_idx = 1
best_val_acc = 0.0
val_accuracies = []
val_f1_scores = []
precisions = []
recalls = []
roc_aucs = []

for train_idx, val_idx in kf.split(dataset):
    print(f"Fold {fold_idx}")
    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    val_dataset = torch.utils.data.Subset(dataset, val_idx)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

    # Initialize model and optimizer
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)
    model = model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # Training loop with early stopping mechanism
    val_acc, val_f1, precision, recall, roc_auc, val_losses, val_f1_scores = train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device, 'bce_loss')
    
    # Store val_acc and val_f1 for current fold
    val_accuracies.append(val_acc)
    val_f1_scores.append(val_f1)
    precisions.append(precision)
    recalls.append(recall)
    roc_aucs.append(roc_auc)

    # Check if current validation accuracy is the best so far
    if val_acc > best_val_acc:
        best_val_acc = val_acc
    
    fold_idx += 1

Fold 1


/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/25] - Train Loss: 0.7034, Train Acc: 95.2609, Val Loss: 0.6951, Val Acc: 0.5053, Val F1: 0.3094, Val Precision: 0.4634, Val Recall: 0.2323, Val ROC-AUC: 0.5208
Validation accuracy improved to 0.5053. Saving model.
Epoch [2/25] - Train Loss: 0.6902, Train Acc: 95.2259, Val Loss: 0.6914, Val Acc: 0.5088, Val F1: 0.6134, Val Precision: 0.4912, Val Recall: 0.8166, Val ROC-AUC: 0.5074
Validation accuracy improved to 0.5088. Saving model.
Epoch [3/25] - Train Loss: 0.6855, Train Acc: 95.2347, Val Loss: 0.6944, Val Acc: 0.5041, Val F1: 0.5176, Val Precision: 0.4831, Val Recall: 0.5575, Val ROC-AUC: 0.5075
Epoch [4/25] - Train Loss: 0.6795, Train Acc: 95.2609, Val Loss: 0.6962, Val Acc: 0.5321, Val F1: 0.3802, Val Precision: 0.5168, Val Recall: 0.3007, Val ROC-AUC: 0.5290
Validation accuracy improved to 0.5321. Saving model.
Epoch [5/25] - Train Loss: 0.6760, Train Acc: 95.2522, Val Loss: 0.6994, Val Acc: 0.5403, Val F1: 0.2423, Val Precision: 0.5676, Val Recall: 0.1540, Val ROC-AUC: 

In [ ]:
print(f"Best Validation Accuracy: {best_val_acc}")
print(f"Validation Accuracies for all folds: {val_accuracies}")
print(f"Validation F1 Scores for all folds: {val_f1_scores}")
print(f"Precision for all folds: {precisions}")
print(f"Recall for all folds: {recalls}")
print(f"ROC AUC for all folds: {roc_aucs}")

# Calculate mean and standard deviation of validation accuracies
mean_val_acc = np.mean(val_accuracies)
std_val_acc = np.std(val_accuracies)

print(f"Mean Validation Accuracy: {mean_val_acc:.4f} +/- {std_val_acc:.4f}")